In [ ]:
!pip install conllu
!pip install gensim

In [ ]:
#FNN POS tagger
import torch
import torch.nn as nn
import torch.optim as optim
from collections import defaultdict
from io import open
import numpy as np
import gensim.downloader as api
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from collections import defaultdict
from io import open
import gensim.downloader as api
file_path = '/content/en_atis-ud-train.conllu'
word2vec_model= api.load("word2vec-google-news-300")
p = 2
s = 2
def parse_conllu_fnn(file_path):
    word_counts = defaultdict(int)
    unique_pos_tags = set()
    with open(file_path, 'r', encoding='utf-8') as data_file:
        for line in data_file:
            if not line.startswith('#') and line.strip():
                parts = line.split('\t')
                if len(parts) > 3:
                    word = parts[1].lower()
                    pos_tag = parts[3]
                    word_counts[word] += 1
                    unique_pos_tags.add(pos_tag)
    return word_counts, unique_pos_tags
def preprocess_conllu_file_fnn(file_path, p, s, low_freq_words):
    input_sequences = []
    labels = []
    with open(file_path, 'r', encoding='utf-8') as data_file:
        sentence_tokens = []
        for line in data_file:
            if line.strip() and not line.startswith('#'):
                parts = line.strip().split('\t')
                if len(parts) > 3:
                    word = parts[1].lower()
                    pos_tag = parts[3]
                    word = word if word not in low_freq_words else '<unknown>'
                    sentence_tokens.append((word, pos_tag))
            elif sentence_tokens:
                words = ['<start>'] + ['<PAD>'] * (p-1) + [token[0] for token in sentence_tokens] + ['<PAD>'] * (s-1) + ['<end>']
                pos_tags = ['<PAD>'] * (p-1) + [token[1] for token in sentence_tokens] + ['<PAD>'] * (s-1)
                for i in range(len(sentence_tokens)):
                    segment_start = i
                    segment_end = i + p + s
                    segment = words[segment_start:segment_end+1]
                    label = pos_tags[i + p - 1]
                    input_sequences.append(segment)
                    labels.append(label)
                sentence_tokens = []
    return input_sequences, labels
def pos_tag_to_one_hot(tag, pos_tag_to_index):
    one_hot_vector = np.zeros(len(pos_tag_to_index))
    one_hot_vector[pos_tag_to_index[tag]] = 1
    return one_hot_vector
def get_word_embedding(word, word2vec_model):
    if word in word2vec_model.key_to_index:
        return word2vec_model[word]
    else:
        return np.zeros(word2vec_model.vector_size)
def preprocess_with_embeddings_and_one_hot(input_sequences, labels, model, pos_tag_to_index):
    embeddings_sequences = []
    one_hot_labels = []
    for sequence in input_sequences:
        embeddings_sequence = [get_word_embedding(word, model) for word in sequence]
        embeddings_sequences.append(embeddings_sequence)
    for label in labels:
        one_hot_label = pos_tag_to_one_hot(label, pos_tag_to_index)
        one_hot_labels.append(one_hot_label)
    return np.array(embeddings_sequences), np.array(one_hot_labels)
def preprocess_sentence_fnn(sentence, model, p, s, low_frequency_words=set('<unknown>')):
    tokens = ['<start>'] + ['<PAD>'] * (p - 1) + sentence.lower().split() + ['<PAD>'] * (s - 1) + ['<end>']
    tokens = [token if token not in low_frequency_words else '<unknown>' for token in tokens]
    embeddings = [get_word_embedding(token, model) for token in tokens]
    input_chunks = []
    for i in range(p, len(tokens) - s):
        context_window = embeddings[i - p:i + s + 1]
        input_chunks.append(context_window)
    return np.array(input_chunks)

def predict_pos_tags_fnn(sentence, model, word2vec_model, pos_tag_to_index, p, s, low_frequency_words):
    input_chunks = preprocess_sentence_fnn(sentence, word2vec_model, p, s, low_frequency_words)
    predicted_pos_tags = []
    for chunk in input_chunks:
        embeddings_flattened = chunk.flatten().reshape(1, -1)
        input_tensor = torch.tensor(embeddings_flattened, dtype=torch.float)
        with torch.no_grad():
            outputs = model(input_tensor)
            prediction = torch.argmax(outputs, dim=1)
        index_to_pos_tag = {v: k for k, v in pos_tag_to_index.items()}
        predicted_pos_tags.append(index_to_pos_tag[prediction.item()])
    return predicted_pos_tags

class POSFFNN(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, output_dim):
        super(POSFFNN, self).__init__()
        self.fc1 = nn.Linear(embedding_dim * (p + s + 1), hidden_dim )
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim , hidden_dim)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.softmax = nn.LogSoftmax(dim=1)
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.softmax(x)
        return x
def pos_with_fnn(sentence):
    word_counts, unique_pos_tags = parse_conllu_fnn(file_path)
    low_frequency_words = {word for word, count in word_counts.items() if count < 3}
    input_sequences, labels = preprocess_conllu_file_fnn(file_path, p, s, low_frequency_words)
    pos_tag_to_index = {tag: idx for idx, tag in enumerate(sorted(unique_pos_tags))}
    model = POSFFNN(word2vec_model.vector_size, 512, len(pos_tag_to_index))
    model = torch.load('/content/pos_ffnn_model .pth')
    model.eval()
    predicted_pos_tags = predict_pos_tags_fnn(sentence, model, word2vec_model, pos_tag_to_index, p, s, set('<unknown>'))
    tokens = sentence.split()
    for token, tag in zip(tokens, predicted_pos_tags):
        print(f'{token} -> {tag}')
sentence = input("Enter a sentence: ")
pos_with_fnn(sentence)

In [ ]:
#LSTM POS tagger
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from collections import defaultdict
import numpy as np
import gensim.downloader as api
from conllu import parse_incr

word2vec_model = api.load("word2vec-google-news-300")
file_path = '/content/en_atis-ud-train.conllu'
word_counts = defaultdict(int)
unique_pos_tags = set()
def parse_conllu(file_path):
    with open(file_path, 'r', encoding='utf-8') as data_file:
        for line in data_file:
            if not line.startswith('#') and line.strip():
                parts = line.split('\t')
                if len(parts) > 3:
                    word = parts[1].lower()
                    pos_tag = parts[3]
                    word_counts[word] += 1
                    unique_pos_tags.add(pos_tag)
parse_conllu(file_path)
vocabulary = {word for word, count in word_counts.items() if count >= 3}
def replace_low_freq_words(sentence):
    return ['<UNK>' if word.lower() not in vocabulary else word.lower() for word in sentence]
def prepare_sequence(seq, to_ix, is_pos=False):
    if is_pos:
        idxs = [to_ix.get(word, to_ix['<UNK>']) for word in seq]
    else:
        idxs = [to_ix.get(word, to_ix['<UNK>']) for word in seq]
    return torch.tensor(idxs, dtype=torch.long)
def get_word_vector(word, model):
    if word in model.key_to_index:
        return model[word]
    else:
        return np.zeros(model.vector_size)
def sentences_to_vectors(padded_sentences, word_to_ix, model):
    sentence_vectors = torch.zeros(padded_sentences.size(0), padded_sentences.size(1), model.vector_size)
    for i, sentence in enumerate(padded_sentences):
        for j, word_idx in enumerate(sentence):
            word = list(word_to_ix.keys())[list(word_to_ix.values()).index(
                word_idx.item())] if word_idx.item() in list(word_to_ix.values()) else "<UNK>"
            word_vector = get_word_vector(word, model)
            sentence_vectors[i, j, :] = torch.tensor(word_vector)
    return sentence_vectors
data_file = open("/content/en_atis-ud-train.conllu", "r", encoding="utf-8")
sentences = []
pos_tags = []
for tokenlist in parse_incr(data_file):
    sentence_tokens = []
    sentence_pos_tags = []
    for token in tokenlist:
        sentence_tokens.append(token['form'])
        sentence_pos_tags.append(token['upos'])
    sentences.append(replace_low_freq_words(sentence_tokens))
    pos_tags.append(sentence_pos_tags)
max_sentence_length = max(len(sentence) for sentence in sentences)
pos_tag_vocab = {tag: idx for idx, tag in enumerate(unique_pos_tags)}
pos_tag_vocab['<PAD>'] = len(pos_tag_vocab)
pos_tag_vocab['<UNK>'] = len(pos_tag_vocab)
word_to_ix = {word: i for i, word in enumerate(vocabulary, start=1)}
word_to_ix['<UNK>'] = 0
word_to_ix['<PAD>'] = len(word_to_ix)
indexed_sentences = [prepare_sequence(sentence, word_to_ix) for sentence in sentences]
indexed_pos_tags = [prepare_sequence(tags, pos_tag_vocab, is_pos=True) for tags in pos_tags]
pad_token = word_to_ix['<PAD>']
pad_tag = pos_tag_vocab['<PAD>']
padded_sentences = pad_sequence(indexed_sentences, batch_first=True, padding_value=pad_token)
padded_pos_tags = pad_sequence(indexed_pos_tags, batch_first=True, padding_value=pad_tag)
sentence_vectors = sentences_to_vectors(padded_sentences, word_to_ix, word2vec_model)
class LSTMPOSTagger(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMPOSTagger, self).__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, num_layers=3, bidirectional=True)
        self.hidden2tag = nn.Linear(hidden_dim * 2, tagset_size)
        self.elu = nn.ELU()
    def forward(self, sentence):
        lstm_out, _ = self.lstm(sentence)
        tag_space = self.hidden2tag(self.elu(lstm_out))
        tag_scores = nn.functional.log_softmax(tag_space, dim=2)
        return tag_scores
model_file_path = '/content/lstm_pos_tagger_model.pth'
checkpoint = torch.load(model_file_path)
loaded_model = LSTMPOSTagger(embedding_dim=checkpoint['input_dim'], hidden_dim=checkpoint['hidden_dim'],vocab_size=len(checkpoint['word_to_ix']),tagset_size=checkpoint['output_dim'])
loaded_model.load_state_dict(checkpoint['model_state_dict'])
word_to_ix = checkpoint['word_to_ix']
pos_tag_vocab = checkpoint['pos_tag_vocab']
loaded_model.eval()
def predict_sentence_tags(sentence, model, word_to_ix, pos_tag_vocab, word2vec_model):
    preprocessed_sentence = replace_low_freq_words(sentence.split())
    indexed_sentence = prepare_sequence(preprocessed_sentence, word_to_ix)
    sentence_padded = pad_sequence([indexed_sentence], batch_first=True, padding_value=word_to_ix['<PAD>'])
    sentence_vectors = sentences_to_vectors(sentence_padded, word_to_ix, word2vec_model)
    with torch.no_grad():
        tag_scores = model(sentence_vectors)
        predicted_tags_idx = torch.argmax(tag_scores, dim=2).squeeze()
    idx_to_tag = {idx: tag for tag, idx in pos_tag_vocab.items()}
    predicted_tags = [idx_to_tag.get(idx.item(), '<UNK>') for idx in predicted_tags_idx]
    return list(zip(sentence.split(), predicted_tags))
sentence = input("Enter a sentence: ")
predicted_tags = predict_sentence_tags(sentence, loaded_model, word_to_ix, pos_tag_vocab, word2vec_model)
for word, tag in predicted_tags:
    print(f'{word} -> {tag}')